In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from google.colab import drive
from tqdm import tqdm
import dill
import numpy as np

In [ ]:
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'rb') as pkl:
    embeddings_dict = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'rb') as pkl:
    ai_word_bank_list = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/topic_dict.pkl', 'rb') as pkl:
    topic_dict = dill.load(pkl)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"

In [ ]:
USE_model = hub.load(module_url)

In [ ]:
def most_similar(phrase: str) -> str :
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    most_similar_value = 0
    most_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase:
            continue
        if np.inner(phrase_embedding, embeddings_dict[word]) > most_similar_value:
            most_similar_value = np.inner(phrase_embedding, embeddings_dict[word])
            most_similar_string = word
    return most_similar_string, most_similar_value

In [ ]:
most_similar("smoothie")

In [ ]:
def build_similarity_dict (topic_dict: dict) -> dict :
    similarity_dict = {}
    for topic in topic_dict:
        similarity_dict[topic] = {}
        topic_literal = str(topic)
        for word in tqdm(ai_word_bank_list):
            if topic == word:
                continue
            else:
                #print(topic)
                similarity_dict[topic][word] = np.inner(embeddings_dict[word],
                                                        embeddings_dict[topic])
        for secret_word in topic_dict[topic]:
            similarity_dict[secret_word] = {}
            for element in tqdm(ai_word_bank_list):
                if secret_word == word:
                    continue
                else:
                    similarity_dict[secret_word][element] = \
                    np.inner(embeddings_dict[secret_word],
                             embeddings_dict[element])
    return similarity_dict

In [ ]:
similarity_dict = build_similarity_dict(topic_dict)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/similarity_dict.pkl', 'wb') as pkl:
    dill.dump(similarity_dict, pkl)

In [ ]:
#this won't make a difference again if this get's adressed earlier in the pipline
delete_small_words_list = []
for word in ai_word_bank_list:
    if len(word) <= 2:
        delete_small_words_list.append(word)

In [ ]:
for word in delete_small_words_list:
    if word in ai_word_bank_list:
        ai_word_bank_list.remove(word)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'wb') as pkl:
    dill.dump(ai_word_bank_list, pkl)

In [ ]:
for word in delete_small_words_list:
    embeddings_dict.pop(word)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'wb') as pkl:
    dill.dump(embeddings_dict, pkl)

In [ ]:
def ai_player_guess(phrase: str) -> dict :
    phrase = phrase.lower()
    guessing_dict = {}
    dict_element_counter = 0
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    minimum_similar_value = 0
    minimum_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase or phrase[:3] in word:
            continue
        if dict_element_counter < 100:
            dict_element_counter += 1
            guessing_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])  
            minimum_similar_string = min(guessing_dict, key=guessing_dict.get)
            #print(minimum_similar_string)
            #print(smoothie_dict[word])
            minimum_similar_value = guessing_dict[minimum_similar_string]            
        
        elif np.inner(phrase_embedding, embeddings_dict[word]) > minimum_similar_value:
            guessing_dict.pop(minimum_similar_string)
            guessing_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])
            minimum_similar_string = min(guessing_dict, key=guessing_dict.get)
            minimum_similar_value = guessing_dict[minimum_similar_string]
    
    sorted_similar_words = sorted(guessing_dict.items(), key=lambda x: x[1],
                                  reverse=True)

    return sorted_similar_words
    
    #for i in sorted_similar_words:
	    #print(i[0], float(i[1]))

In [ ]:
ai_player_guess('Math')

In [ ]:
def score_calibrator(phrase: str):
    high_list = []
    low_list = []
    fine_list = []
    sorted_similar_words = ai_player_guess(phrase)
    for i in sorted_similar_words:
        choice =input(f"Is *{i[0]}* with a score of {i[1]} too (h)ighly related,"\
              f" (l)lowly related, or just (f)ine?")
        if choice == 'h':
            high_list.append(i)
            print(f"You added {i[0]} with {i[1]} as high similarity.")
        elif choice == 'l':
            low_list.append(i)
            print(f"You added {i[0]} with {i[1]} as low similarity.")
        elif choice == 'f':
            print(f"You added {i[0]} with {i[1]} as fine similarity.")
            fine_list.append(i)
        else:
            print(f"Error. {choice} is an invalid command.")
    return high_list, low_list, fine_list

In [ ]:
high_list, low_list, fine_list = score_calibrator("Steam Engine")

In [ ]:
low_list

In [ ]:
high_list

In [ ]:
fine_list

In [ ]:
gunpowder_string = 'gun powder'
gunfight_string = 'gun fight'
fightnight_string = 'fight night'

In [ ]:
if gunpowder_string[:3] in gunfight_string:
    print('too close')

In [ ]:
def string_checker(string1, string2):
    if len(string1) >= 5 and len(string2) >= 5:
        for index, letter in enumerate(string1):
            if index <= (len(string1) - 5):
                sub_string = string1[index : index + 5]
                print(sub_string)
                if sub_string in string2:
                    print("it's a trap")
                    break

In [ ]:
string_checker('gunfight', 'fight night')